In [1]:
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csc_matrix
import numpy as np
from Algorithm import CoordinateDescent

In [2]:
%%time 
X_train, y_train = load_svmlight_file("data/rcv1_train.binary")
X_train = csc_matrix(X_train)

cd = CoordinateDescent(C=1., eps=1e-3, max_iter=1)

w = cd.fit_process(X_train, y_train)

2023-06-16T20:31:12.909738 0 20242.0
CPU times: user 3min 16s, sys: 7min 22s, total: 10min 39s
Wall time: 1min 3s


In [4]:
%load_ext snakeviz

In [5]:
%%snakeviz
X_train, y_train = load_svmlight_file("data/rcv1_train.binary")
X_train = csc_matrix(X_train)

cd = CoordinateDescent(C=1., eps=1e-3, max_iter=1)

w = cd.fit_process(X_train, y_train)

2023-06-16T20:32:25.929445 0 20242.0
 
*** Profile stats marshalled to file '/tmp/tmp4k_ym1nz'.
Embedding SnakeViz in this document...
<function display at 0x7f4c616d2680>


In [2]:
from line_profiler import LineProfiler

In [18]:
type(X_train.multiply(y_train[None].T)
    )

scipy.sparse._coo.coo_matrix

In [9]:
y_train

array([ 1.,  1.,  1., ..., -1.,  1., -1.])

In [15]:
X_train[:, 1].multiply(y_train[None].T)

<20242x1 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in COOrdinate format>

In [12]:
X_train.shape

(20242, 47236)

In [17]:
X_train.multiply(X_train)

<20242x47236 sparse matrix of type '<class 'numpy.float64'>'
	with 1498952 stored elements in Compressed Sparse Column format>

In [52]:
idx = np.random.randint(2, size=X_train.shape[0]) == 1
original = np.zeros(X_train.shape[1])
speed1 = np.zeros(X_train.shape[1])
speed2 = np.zeros(X_train.shape[1])

In [53]:
%%time
for i in range(X_train.shape[1]):
    a = np.sum(X_train[idx, i])
    original[i] = a

CPU times: user 11.1 s, sys: 0 ns, total: 11.1 s
Wall time: 11.1 s


In [54]:
%%time
for i in range(X_train.shape[1]):
    total = 0
    col_slice = X_train[:, i]
    for data, el in zip(col_slice.data, col_slice.indices):
        if idx[el]:
            total += data
    speed1[i] = total

CPU times: user 2.52 s, sys: 55.1 ms, total: 2.57 s
Wall time: 2.56 s


In [55]:
%%time
for i in range(X_train.shape[1]):
    total = 0
    start = X_train.indptr[i]
    end = X_train.indptr[i+1]
    col_sliced = X_train.data[start:end]
    col_slicei = X_train.indices[start:end]
    for data, el in zip(col_sliced, col_slicei):
        if idx[el]:
            total += data
    speed2[i] = total

CPU times: user 324 ms, sys: 0 ns, total: 324 ms
Wall time: 324 ms


In [57]:
np.allclose(original, speed1), np.allclose(original, speed2)

(True, True)

In [49]:
%%snakeviz
for i in range(X_train.shape[1]):
    total = 0
    col_slice = X_train[:, i]
    for data, el in zip(col_slice.data, col_slice.indices):
        if idx[el]:
            total += data

 
*** Profile stats marshalled to file '/tmp/tmp5dcmzgd3'.
Embedding SnakeViz in this document...
<function display at 0x7f4c616d2680>


In [48]:
%%snakeviz
for i in range(X_train.shape[1]):
    total = 0
    start = X_train.indptr[i]
    end = X_train.indptr[i+1]
    col_sliced = X_train.data[start:end]
    col_slicei = X_train.indices[start:end]
    for data, el in zip(col_sliced, col_slicei):
        if idx[el]:
            total += data

 
*** Profile stats marshalled to file '/tmp/tmpc519xlfh'.
Embedding SnakeViz in this document...
<function display at 0x7f4c616d2680>
